# 子查詢抓取
1. 擷取所有Node list 
2. loop所有Node
3. 細節如下 
 - 若整體無keyword return “ ” (應該不用)
 - loop : token_next()功能抓取下一個token 判斷有沒有 "括號" 。
 - 若有keyword token ： 更新dict
 - 記錄功能列 : SELECT 、 WHERE 、FROM
 - 遇到Where 要往下伸展
 - 判斷括號進入 : subquery_trace(dict, sqlfun, tokens)
 - 迴圈結束後return dictionary

In [53]:
import sqlparse

In [57]:
the_query = '''CREATE TABLE WORK.FINALTABLE as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 , BBB_table as t2
                    where t3.X1 = 'thing' and t3.X2 in ('a1', 'a2', 'a3') 
               Group by a1
               limit 15'''

In [58]:
res = sqlparse.parse(the_query)
stmt = res[0]
stmt._pprint_tree() 

|- 0 DDL 'CREATE'
|- 1 Whitespace ' '
|- 2 Keyword 'TABLE'
|- 3 Whitespace ' '
|- 4 Identifier 'WORK.F...'
|  |- 0 Name 'WORK'
|  |- 1 Punctuation '.'
|  `- 2 Name 'FINALT...'
|- 5 Whitespace ' '
|- 6 Keyword 'as'
|- 7 Whitespace ' '
|- 8 Newline ' '
|- 9 Whitespace ' '
|- 10 Whitespace ' '
|- 11 Whitespace ' '
|- 12 Whitespace ' '
|- 13 Whitespace ' '
|- 14 Whitespace ' '
|- 15 Whitespace ' '
|- 16 Whitespace ' '
|- 17 Whitespace ' '
|- 18 Whitespace ' '
|- 19 Whitespace ' '
|- 20 Whitespace ' '
|- 21 Whitespace ' '
|- 22 Whitespace ' '
|- 23 Whitespace ' '
|- 24 DML 'select'
|- 25 Whitespace ' '
|- 26 IdentifierList 'a1, co...'
|  |- 0 Identifier 'a1'
|  |  `- 0 Name 'a1'
|  |- 1 Punctuation ','
|  |- 2 Whitespace ' '
|  `- 3 Identifier 'count'
|     `- 0 Name 'count'
|- 27 Parenthesis '(a2)'
|  |- 0 Punctuation '('
|  |- 1 Identifier 'a2'
|  |  `- 0 Name 'a2'
|  `- 2 Punctuation ')'
|- 28 Whitespace ' '
|- 29 Newline ' '
|- 30 Whitespace ' '
|- 31 Whitespace ' '
|- 32 Whitespace ' '

In [59]:
print("======CREATE ==============")
print(stmt.tokens[0].value) # Select
print(stmt.tokens[0].ttype)
print(stmt.tokens[0].__str__)
print("======TABLE ==============")
print(stmt.tokens[2].value) # Select
print(stmt.tokens[2].ttype)
print(stmt.tokens[2].__str__)
print("======TABLE NAME(identifier) ==============")
print(stmt.tokens[4].value) # Select
print(stmt.tokens[4].ttype)
print(stmt.tokens[4].__str__)
print("======Select ==============")
print(stmt.tokens[24].value) # Select
print(stmt.tokens[24].ttype)
print(stmt.tokens[24].__str__)
print("======Select 的文字==============")
print(stmt.tokens[26].value) # Select 的文字
print(stmt.tokens[26].ttype)
print(stmt.tokens[26].__str__)
print("======FROM ==============")
print(stmt.tokens[45].value)
print(stmt.tokens[45].ttype)
print(stmt.tokens[45].__str__)
print("======FROM 的文字 ==============")
print(stmt.tokens[47].value)
print(stmt.tokens[47].ttype)
print(stmt.tokens[47].__str__)
print("======WHERE ==============")
print(stmt.tokens[69].value)
print(stmt.tokens[69].ttype)
print(stmt.tokens[69].__str__)

======CREATE ==============
CREATE
Token.Keyword.DDL
<bound method Token.__str__ of <DDL 'CREATE' at 0x5322F48>>
======TABLE ==============
TABLE
Token.Keyword
<bound method Token.__str__ of <Keyword 'TABLE' at 0x5334AC8>>
======TABLE NAME(identifier) ==============
TABLE
Token.Keyword
<bound method Token.__str__ of <Keyword 'TABLE' at 0x5334AC8>>
======Select ==============
select
Token.Keyword.DML
<bound method Token.__str__ of <DML 'select' at 0x5346468>>
======Select 的文字==============
a1, count
None
<bound method TokenList.__str__ of <IdentifierList 'a1, co...' at 0x5339408>>
======FROM ==============
from
Token.Keyword
<bound method Token.__str__ of <Keyword 'from' at 0x5346E28>>
======FROM 的文字 ==============
MEOW.AAA_table as t3 , BBB_table as t2
None
<bound method TokenList.__str__ of <IdentifierList 'MEOW.A...' at 0x5339480>>
======WHERE ==============
where t3.X1 = 'thing' and t3.X2 in ('a1', 'a2', 'a3') 
               
None
<bound method TokenList.__str__ of <Where 'where ..

# LEFT JOIN 測試

In [38]:
the_query2 = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join MEOW.BBB_table as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in ('a1', 'a2', 'a3') 
               Group by a1
               limit 15'''

In [41]:
res2 = sqlparse.parse(the_query2)
stmt2 = res2[0]
stmt2._pprint_tree() 

|- 0 DDL 'CREATE'
|- 1 Whitespace ' '
|- 2 Keyword 'TABLE'
|- 3 Whitespace ' '
|- 4 Keyword 'FINAL'
|- 5 Whitespace ' '
|- 6 Keyword 'as'
|- 7 Whitespace ' '
|- 8 Newline ' '
|- 9 Whitespace ' '
|- 10 Whitespace ' '
|- 11 Whitespace ' '
|- 12 Whitespace ' '
|- 13 Whitespace ' '
|- 14 Whitespace ' '
|- 15 Whitespace ' '
|- 16 Whitespace ' '
|- 17 Whitespace ' '
|- 18 Whitespace ' '
|- 19 Whitespace ' '
|- 20 Whitespace ' '
|- 21 Whitespace ' '
|- 22 Whitespace ' '
|- 23 Whitespace ' '
|- 24 DML 'select'
|- 25 Whitespace ' '
|- 26 IdentifierList 'a1, co...'
|  |- 0 Identifier 'a1'
|  |  `- 0 Name 'a1'
|  |- 1 Punctuation ','
|  |- 2 Whitespace ' '
|  `- 3 Identifier 'count'
|     `- 0 Name 'count'
|- 27 Parenthesis '(a2)'
|  |- 0 Punctuation '('
|  |- 1 Identifier 'a2'
|  |  `- 0 Name 'a2'
|  `- 2 Punctuation ')'
|- 28 Whitespace ' '
|- 29 Newline ' '
|- 30 Whitespace ' '
|- 31 Whitespace ' '
|- 32 Whitespace ' '
|- 33 Whitespace ' '
|- 34 Whitespace ' '
|- 35 Whitespace ' '
|- 36 Whites

In [47]:
print("======Select ==============")
print(stmt2.tokens[24].value) # Select
print(stmt2.tokens[24].ttype)
print(stmt2.tokens[24].__str__)
print("======Select 的文字==============")
print(stmt2.tokens[26].value) # Select 的文字
print(stmt2.tokens[26].ttype)
print(stmt2.tokens[26].__str__)
print("======FROM ==============")
print(stmt2.tokens[45].value)
print(stmt2.tokens[45].ttype)
print(stmt2.tokens[45].__str__)
print("======FROM 的文字(identifier) ==============")
print(stmt2.tokens[47].value)
print(stmt2.tokens[47].ttype)
print(stmt2.tokens[47].__str__)
print("======left join ==============")
print(stmt2.tokens[70].value)
print(stmt2.tokens[70].ttype)
print(stmt2.tokens[70].__str__)
print("======left join 的文字(identifier) ==============")
print(stmt2.tokens[72].value)
print(stmt2.tokens[72].ttype)
print(stmt2.tokens[72].__str__)
print("======WHERE ==============")
print(stmt2.tokens[93].value)
print(stmt2.tokens[93].ttype)
print(stmt2.tokens[93].__str__)

======Select ==============
select
Token.Keyword.DML
<bound method Token.__str__ of <DML 'select' at 0x53171C8>>
======Select 的文字==============
a1, count
None
<bound method TokenList.__str__ of <IdentifierList 'a1, co...' at 0x531A048>>
======FROM ==============
from
Token.Keyword
<bound method Token.__str__ of <Keyword 'from' at 0x5317B88>>
======FROM 的文字(identifier) ==============
MEOW.AAA_table as t3
None
<bound method TokenList.__str__ of <Identifier 'MEOW.A...' at 0x530C930>>
======left join ==============
left join
Token.Keyword
<bound method Token.__str__ of <Keyword 'left j...' at 0x531D768>>
======left join 的文字(identifier) ==============
MEOW.BBB_table as t2
None
<bound method TokenList.__str__ of <Identifier 'MEOW.B...' at 0x530C9A8>>
======WHERE ==============
where t3.X1 = 'thing' and t3.X2 in ('a1', 'a2', 'a3') 
               
None
<bound method TokenList.__str__ of <Where 'where ...' at 0x530C840>>


# 子查詢測試
子查詢的table會跟之前的不一樣，t2的標註無法明確標註，可能要將原 (SELECT...) 取代後再parse一次。

In [65]:
the_query3 = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join (SELECT * FROM MEOW.BBB_table ) as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
               Group by a1
               limit 15'''

In [66]:
res3 = sqlparse.parse(the_query3)
stmt3 = res3[0]
stmt3._pprint_tree() 

|- 0 DDL 'CREATE'
|- 1 Whitespace ' '
|- 2 Keyword 'TABLE'
|- 3 Whitespace ' '
|- 4 Keyword 'FINAL'
|- 5 Whitespace ' '
|- 6 Keyword 'as'
|- 7 Whitespace ' '
|- 8 Newline ' '
|- 9 Whitespace ' '
|- 10 Whitespace ' '
|- 11 Whitespace ' '
|- 12 Whitespace ' '
|- 13 Whitespace ' '
|- 14 Whitespace ' '
|- 15 Whitespace ' '
|- 16 Whitespace ' '
|- 17 Whitespace ' '
|- 18 Whitespace ' '
|- 19 Whitespace ' '
|- 20 Whitespace ' '
|- 21 Whitespace ' '
|- 22 Whitespace ' '
|- 23 Whitespace ' '
|- 24 DML 'select'
|- 25 Whitespace ' '
|- 26 IdentifierList 'a1, co...'
|  |- 0 Identifier 'a1'
|  |  `- 0 Name 'a1'
|  |- 1 Punctuation ','
|  |- 2 Whitespace ' '
|  `- 3 Identifier 'count'
|     `- 0 Name 'count'
|- 27 Parenthesis '(a2)'
|  |- 0 Punctuation '('
|  |- 1 Identifier 'a2'
|  |  `- 0 Name 'a2'
|  `- 2 Punctuation ')'
|- 28 Whitespace ' '
|- 29 Newline ' '
|- 30 Whitespace ' '
|- 31 Whitespace ' '
|- 32 Whitespace ' '
|- 33 Whitespace ' '
|- 34 Whitespace ' '
|- 35 Whitespace ' '
|- 36 Whites

In [62]:
print("======CREATE ==============")
print(stmt.tokens[0].value) # Select
print(stmt.tokens[0].ttype)
print(stmt.tokens[0].__str__)
print("======TABLE ==============")
print(stmt.tokens[2].value) # Select
print(stmt.tokens[2].ttype)
print(stmt.tokens[2].__str__)
print("======TABLE NAME(identifier) ==============")
print(stmt.tokens[4].value) # Select
print(stmt.tokens[4].ttype)
print(stmt.tokens[4].__str__)
print("======Select ==============")
print(stmt3.tokens[24].value.upper()) # Select
print(stmt3.tokens[24].ttype)
print(stmt3.tokens[24].__str__)
print("======Select 的文字==============")
print(stmt3.tokens[26].value) # Select 的文字
print(stmt3.tokens[26].ttype)
print(stmt3.tokens[26].__str__)
print("======FROM ==============")
print(stmt3.tokens[45].value)
print(stmt3.tokens[45].ttype)
print(stmt3.tokens[45].__str__)
print("======FROM 的文字(identifier) ==============")
print(stmt3.tokens[47].value)
print(stmt3.tokens[47].ttype)
print(stmt3.tokens[47].__str__)
print("======left join ==============")
print(stmt3.tokens[70].value)
print(stmt3.tokens[70].ttype)
print(stmt3.tokens[70].__str__)
print("======left join 的文字(identifier) ==============")
print(stmt3.tokens[72].value)
print(stmt3.tokens[72].ttype)
print(stmt3.tokens[72].__str__)
print("======left join 的文字(Parenthesis ) ==============")
print(stmt3.tokens[76].value)
print(stmt3.tokens[76].ttype)
print(stmt3.tokens[76].__str__)
print("======WHERE ==============")
print(stmt3.tokens[93].value)
print(stmt3.tokens[93].ttype)
print(stmt3.tokens[93].__str__)

======CREATE ==============
CREATE
Token.Keyword.DDL
<bound method Token.__str__ of <DDL 'CREATE' at 0x5322F48>>
======TABLE ==============
TABLE
Token.Keyword
<bound method Token.__str__ of <Keyword 'TABLE' at 0x5334AC8>>
======TABLE NAME(identifier) ==============
WORK.FINALTABLE
None
<bound method TokenList.__str__ of <Identifier 'WORK.F...' at 0x5340DE0>>
======Select ==============
SELECT
Token.Keyword.DML
<bound method Token.__str__ of <DML 'select' at 0x5315AC8>>
======Select 的文字==============
a1, count
None
<bound method TokenList.__str__ of <IdentifierList 'a1, co...' at 0x531AF48>>
======FROM ==============
from
Token.Keyword
<bound method Token.__str__ of <Keyword 'from' at 0x53254C8>>
======FROM 的文字(identifier) ==============
MEOW.AAA_table as t3
None
<bound method TokenList.__str__ of <Identifier 'MEOW.A...' at 0x531A6D8>>
======left join ==============
left join
Token.Keyword
<bound method Token.__str__ of <Keyword 'left j...' at 0x53280A8>>
======left join 的文字(identifier